In [118]:
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (3,610 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120875 files and dire

In [ ]:
!pip install konlpy
from konlpy.tag import Kkma
tokenizer=Kkma()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 38.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [119]:
file_path = '/content/drive/MyDrive/Colab Notebooks/TextMining/기말과제/대통령_취임사.csv'


In [138]:
import pandas as pd
# CSV 파일 읽기
df = pd.read_csv(file_path)
# 특정 열 두 개 선택
selected_df = df[['대통령', '연설내용']] # 실제 열 이름으로 수정 필요


In [139]:
from konlpy.tag import Kkma
kkma = Kkma()

# NaN 값을 제거
selected_df = selected_df.dropna(subset=['연설내용'])
# '연설내용' 열을 문자열로 변환
selected_df['연설내용'] = selected_df['연설내용'].astype(str)
# 특수 문자 제거 (한글과 공백만 남기기)
selected_df['연설내용'] = selected_df['연설내용'].apply(lambda x: re.sub('[^가-힣\s]', '', x))
# '대통령' 열을 기준으로 데이터 그룹화
grouped = selected_df.groupby('대통령')


In [ ]:
# 각 대통령별 연설문 토크나이징
tokenized_data = {}
for president, group in grouped:
    tokenized_texts = group['연설내용'].apply(lambda x: kkma.morphs(x))
    tokenized_data[president] = tokenized_texts.tolist()
# 결과 출력
for president, tokens in tokenized_data.items():
    print(f'대통령: {president}')
    for token_list in tokens:
        print(token_list)

대통령: 김대중
['존경', '하', '고', '사랑', '하', '는', '국민', '여러분', '오늘', '저', '는', '대한민국', '제대', '대통령', '에', '취임', '하', '게', '되', '었', '습니다', '정부', '수립', '년', '만', '에', '처음', '이루어지', 'ㄴ', '여야', '간', '정권', '교체', '를', '여러분', '과', '함께', '기뻐하', '면서', '온갖', '시련', '과', '장벽', '을', '넘', '어', '진정', '하', 'ㄴ', '국민', '의', '정부', '를', '탄생', '시키', 'ㄴ', '국민', '여러분', '께', '찬양', '과', '감사', '의', '말씀', '을', '드리', '는', '바', '이', 'ㅂ니다', '그리고', '저의', '취임', '을', '축하', '하', '기', '위하', '어', '이', '자리', '에', '함께', '하', '어', '주신', '김영삼', '전임', '대통령', '폰', '바', '이체', '커', '어', '독일', '전', '대통령', '코', '라', '손', '아키노', '필리핀', '전', '대통령', '후안', '안', '토니오', '사마', '란', '치', '위원장', '등', '내외', '귀빈', '을', '비롯', '하', 'ㄴ', '참석자', '여러분', '께', '도', '깊이', '감사', '드리', 'ㅂ니다', '오늘', '이', '취임식', '의', '역사적', '이', 'ㄴ', '의미', '는', '참으로', '크', '다고', '하', 'ㄹ', '것', '이', 'ㅂ니다', '오늘', '은', '이', '땅', '에서', '처음', '으로', '민주적', '정권', '교체', '가', '실현', '되', '는', '자랑', '스럽', 'ㄴ', '날', '이', 'ㅂ니다', '또', '하', 'ㄴ', '민주주의', '와', '경제', '를', '동시', '에', '발전', '시키', 

In [137]:
# 파일에서 불용어 불러오기
file_path1 = '/content/drive/MyDrive/Colab Notebooks/TextMining/기말과제/한글 불용어.txt'
with open(file_path1, 'r', encoding='utf-8') as f:
    file_stopwords = [line.strip() for line in f.readlines()]
# 기본 불용어
basic_stopwords = ['하', '의', '을', '에', '는', '이', '를', '과', '도', '와', '으로', '에서', '위하','대하','시다', '로', '에게', '라', '만', '게','고자', '로서', '읍니', '대해']
# 두 불용어 리스트 합치기
stopwords = file_stopwords + basic_stopwords
# 중복된 불용어 제거
stopwords = list(set(stopwords))

In [ ]:
from collections import Counter
pos_frequencies = {}
# 대통령별로 토큰과 해당 토큰의 품사를 묶어서 빈도 계산
for president, token_lists in tokenized_data.items():
    token_pos_counts = Counter()
    for tokens in token_lists:
        pos_tagged_tokens = kkma.pos(" ".join(tokens))
        # 길이가 두 글자 이상이며 불용어에 속하지 않는 토큰만 포함
        filtered_tokens = [
            (token, pos) for token, pos in pos_tagged_tokens
            if len(token) >= 2 and token not in stopwords
        ]
        token_pos_counts.update(filtered_tokens)
    sorted_pos_counts = sorted(token_pos_counts.items(), key=lambda x: x[1], reverse=True)
    pos_frequencies[president] = sorted_pos_counts

# 결과 출력
for president, sorted_pos_counts in pos_frequencies.items():
    for (token, pos), freq in sorted_pos_counts:
        print(f'대통령: {president}, 단어: {token}, 품사: {pos}, 빈도: {freq}')


In [ ]:
# 대통령별 상위 20개 결과 출력
for president, sorted_pos_counts in pos_frequencies.items():
    print(f'대통령: {president}')
    for (token, pos), freq in sorted_pos_counts[:20]:  # 상위 20개만 출력
        print(f'단어: {token}, 품사: {pos}, 빈도: {freq}')
    print("-" * 50)  # 구분선


In [ ]:
top_nouns_by_president = {}
# 각 대통령별로 'N'으로 시작하는 품사의 토큰만 상위 20개 선택
for president, sorted_pos_counts in pos_frequencies.items():
    top_nouns = [(token, freq) for (token, pos), freq in sorted_pos_counts if pos.startswith('N')]
    top_nouns_by_president[president] = top_nouns

# 결과 출력
for president, top_nouns in top_nouns_by_president.items():
    print(f'대통령: {president}')
    for token, freq in top_nouns:
        print(f'단어: {token}, 빈도: {freq}')
    print('-' * 50)  # 구분선

In [ ]:
!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=font_path, size=10).get_name()
plt.rc('font', family=font_name)

for president, top_nouns in top_nouns_by_president.items():
    wc = WordCloud(
        font_path=font_path,  # 폰트 경로
        background_color='white',  # 배경색 설정
        width=300,
        height=300
    )

    # 워드클라우드 생성
    wordcloud = wc.generate_from_frequencies(dict(top_nouns))

    # 그래프 설정 및 표시
    plt.figure(figsize=(5, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'대통령: {president}')
    plt.axis('off')
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=path, size=5)
plt.rc('font', family='NanumGothic')

for president, top_nouns in top_nouns_by_president.items():
    plt.figure(figsize=(8, 5))  # 그래프 크기 설정
    # 데이터 분리
    words, frequencies = zip(*top_nouns)
    # 그래프 그리기
    sns.barplot(x=list(frequencies), y=list(words))
    plt.title(f"Top 20 Nouns used by {president}")  # 제목 설정
    plt.xlabel('Frequency')  # x축 라벨 설정
    plt.ylabel('Words')      # y축 라벨 설정
    plt.show()

TF-IDF vectorize -- 김영빈

In [ ]:
# 아래 작업들은 완료되었다고 가정하고 selected_df 에서 출발

# # NaN 값을 제거
# selected_df = selected_df.dropna(subset=['연설내용'])
# # '연설내용' 열을 문자열로 변환
# selected_df['연설내용'] = selected_df['연설내용'].astype(str)
# # 특수 문자 제거 (한글과 공백만 남기기)
# selected_df['연설내용'] = selected_df['연설내용'].apply(lambda x: re.sub('[^가-힣\s]', '', x))


# # '대통령' 열을 기준으로 데이터 그룹화
# grouped = selected_df.groupby('대통령')

In [140]:
# # '대통령' 열을 기준으로 데이터 그룹화  -- 이승만, 박정희가 여러 번 나오는데 시대상이 반영될 수도 있으므로 우선 적용하지 않음
# grouped1 = df.groupby('대통령')['연설내용'].agg(lambda x: ', '.join(x))

df_raw = selected_df.copy()
print(df_raw.head(5))
print(df_raw.shape)  # (21, 2)

In [141]:
from konlpy.tag import Okt

# KoNLPy의 Okt 형태소 분석기 초기화
okt = Okt()

# 품사 태깅, 불용어 제거 및 명사 추출 함수 정의
def process_text(text):
    tokens = okt.pos(text)
    nouns = [token[0] for token in tokens if token[1] == 'Noun' and len(token[0]) >= 2]
    nouns = [word for word in nouns if word not in stopwords]
    return nouns

In [142]:
df_raw['연설내용'] = df_raw['연설내용'].apply(process_text)

In [143]:
print(df_raw.head(5))

df_raw.iloc[20,]

   대통령                                               연설내용
0  이승만  [여러, 하나님, 은혜, 동포, 애호, 지금, 오늘, 영광, 추대, 나로, 일변, ...
1  이승만  [오늘, 취임식, 다시, 지게, 책임, 수만, 지지, 정부, 수업, 사람, 국민, ...
2  이승만  [사랑, 동포, 오늘, 한번, 민중, 서서, 대통령, 취임, 선언, 차로, 동포, ...
3  박정희  [단군, 조가, 천혜, 강토, 국기, 신지, 반만년, 역사, 전통, 이제, 공화국,...
4  박정희  [단군, 조가, 천혜, 강토, 국기, 신지, 반만년, 역사, 전통, 이제, 대한민국...


대통령                                                   문재인
연설내용    [존경, 국민, 임기, 보통, 라면, 마무리, 생각, 시점, 지난, 위기, 국민, ...
Name: 20, dtype: object

In [144]:
# 3) TF-IDF 행렬 만들기
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(df_raw)):
  t = ' '.join(df_raw['연설내용'][i])
  detokenized_doc.append(t)

# 다시 df_raw['연설내용']에 재저장
df_raw['연설내용'] = detokenized_doc
# 5개의 샘플을 출력
df_raw['연설내용'][:5]

0    여러 하나님 은혜 동포 애호 지금 오늘 영광 추대 나로 일변 감격 마음 일변 책임 ...
1    오늘 취임식 다시 지게 책임 수만 지지 정부 수업 사람 국민 정부 관료 평민 나라 ...
2    사랑 동포 오늘 한번 민중 서서 대통령 취임 선언 차로 동포 지도 신념 표시 마음 ...
3    단군 조가 천혜 강토 국기 신지 반만년 역사 전통 이제 공화국 국헌 준수 신명 조국...
4    단군 조가 천혜 강토 국기 신지 반만년 역사 전통 이제 대한민국 대통령 취임 국헌 ...
Name: 연설내용, dtype: object

In [145]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [146]:
# 사이킷런의 TfidfVectorizer를 이용한 TF-IDF 행렬 만들기
# 간단히 1,000개의 단어로 제한
# 상위 1,000개의 단어를 보존
vectorizer = TfidfVectorizer(stop_words = stopwords, max_features= 1000)
X = vectorizer.fit_transform(df_raw['연설내용'])
# TF-IDF 행렬의 크기 확인: (1244184, 1000)
print('TF-IDF 행렬의 크기 :',X.shape)

TF-IDF 행렬의 크기 : (21, 1000)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '만큼', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '미치다', '바꾸어서', '바꿔', '밖에', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '어찌됏든', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하다', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn(


In [ ]:
print(X)
print(type(X))

<class 'scipy.sparse._csr.csr_matrix'>


In [147]:
feature_names = vectorizer.get_feature_names_out()

In [ ]:
print(feature_names)
type(feature_names)

numpy.ndarray

In [148]:
import numpy as np

In [149]:
# 각 행별로 최대값에 해당하는 열 위치(인덱스)를 구하는 함수
def row_argmax(matrix):
    argmax_indices = np.argmax(matrix, axis=1)  # 각 행별로 최대값의 인덱스
    return argmax_indices

In [150]:
# 각 행별로 최대값을 구하는 함수
def row_max(matrix):
    max_values = np.amax(matrix, axis=1)  # 각 행별로 최대값
    return max_values

In [152]:
# print(row_argmax(X))
row_argmax = row_argmax(X)
# type(row_argmax)
# row_argmax.shape
# row_argmax[0][0].shape
# row_argmax[0].shape
# row_argmax[0]

In [156]:
## 가장 중요한 단어 찾기

for i in range(len(df_raw)):
  president = df_raw.iloc[i, 0]
  argmax = row_argmax[i]
  imp_feature = feature_names[argmax]
  print(f'{i+1}대 대통령: {president}, 중요단어: {imp_feature}')

1대 대통령: 이승만, 중요단어: [['친선']]
2대 대통령: 이승만, 중요단어: [['원수']]
3대 대통령: 이승만, 중요단어: [['농민']]
4대 대통령: 박정희, 중요단어: [['공화국']]
5대 대통령: 박정희, 중요단어: [['근대화']]
6대 대통령: 박정희, 중요단어: [['조국']]
7대 대통령: 박정희, 중요단어: [['유신']]
8대 대통령: 박정희, 중요단어: [['국민']]
9대 대통령: 박정희, 중요단어: [['국민']]
10대 대통령: 최규하, 중요단어: [['헌법']]
11대 대통령: 전두환, 중요단어: [['국민']]
12대 대통령: 전두환, 중요단어: [['본인']]
13대 대통령: 노태우, 중요단어: [['국민']]
14대 대통령: 김영삼, 중요단어: [['국민']]
15대 대통령: 김대중, 중요단어: [['국민']]
16대 대통령: 노무현, 중요단어: [['도약']]
17대 대통령: 노무현, 중요단어: [['동북아']]
18대 대통령: 이명박, 중요단어: [['선진']]
19대 대통령: 박근혜, 중요단어: [['국민']]
20대 대통령: 문재인, 중요단어: [['대통령']]
21대 대통령: 문재인, 중요단어: [['코로나']]


In [154]:
print(row_max(X))

  (0, 0)	0.3437272657411849
  (1, 0)	0.3330940514314938
  (2, 0)	0.3698712419707421
  (3, 0)	0.3345122175213323
  (4, 0)	0.2335523284920976
  (5, 0)	0.22598359121564293
  (6, 0)	0.5098990026036078
  (7, 0)	0.3634820840259793
  (8, 0)	0.31188038011239233
  (9, 0)	0.3773410898438021
  (10, 0)	0.2546353338050965
  (11, 0)	0.5846933317530804
  (12, 0)	0.48719914355183536
  (13, 0)	0.4134741255096121
  (14, 0)	0.3656134970316945
  (15, 0)	0.30648593026476506
  (16, 0)	0.5012287661528341
  (17, 0)	0.275130614019106
  (18, 0)	0.5362288158193855
  (19, 0)	0.5559456493260639
  (20, 0)	0.4464773348907474


In [162]:
### n 번째로 중요한 단어 가져오기 -- 실패

# def row_arg_nst(matrix, order):
#   nth_largest_indices = []
#   for row in matrix:
#     # 행별로 내림차순으로 정렬 후 인덱스를 가져오기
#     sorted_indices = np.argsort(row)[::-1]
#     # 두 번째로 큰 값의 인덱스 찾기
#     nth_largest_index = sorted_indices[order-1]
#     # 결과 리스트에 추가
#     nth_largest_indices.append(nth_largest_index)
#   return nth_largest_indices